<a href="https://colab.research.google.com/github/Neel7317/Generative-Models/blob/main/Adapter_Based_Fine_Tuning_for_DistilBERT_with_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig

# Sample Dataset
# Multi-class classification dataset with labels: Sports (0), Politics (1), Education (2)
data = [
    {"text": "The soccer team clinched a thrilling victory in overtime.", "label": 0},
    {"text": "The cricket championship finals attracted global attention.", "label": 0},
    {"text": "A new record was set in the 100-meter sprint.", "label": 0},
    {"text": "Tennis legends face off in a historic grand slam final.", "label": 0},
    {"text": "The basketball league announces the upcoming season schedule.", "label": 0},
    {"text": "Fans cheer as their favorite football club wins the title.", "label": 0},
    {"text": "Olympic gold medalist announces retirement from the sport.", "label": 0},
    {"text": "The marathon saw record participation this year.", "label": 0},
    {"text": "Baseball's star player hits a remarkable home run.", "label": 0},
    {"text": "The Formula 1 race witnessed a dramatic last-lap overtake.", "label": 0},
    {"text": "New training facilities unveiled for the national hockey team.", "label": 0},
    {"text": "The underdog team surprises everyone with a stunning win.", "label": 0},
    {"text": "The swimming championship showcased extraordinary talent.", "label": 0},
    {"text": "The coach shares strategies ahead of the semi-finals.", "label": 0},
    {"text": "Athletes gear up for the international sports meet.", "label": 0},
    {"text": "The volleyball team secures a crucial win in the tournament.", "label": 0},
    {"text": "The government unveils a new healthcare policy.", "label": 1},
    {"text": "The president addresses the nation on economic reforms.", "label": 1},
    {"text": "The election results spark debates across the country.", "label": 1},
    {"text": "Parliament passes a bill to boost renewable energy.", "label": 1},
    {"text": "Diplomatic talks between the two nations yield progress.", "label": 1},
    {"text": "The mayor announces plans for urban development.", "label": 1},
    {"text": "The opposition criticizes the new tax policies.", "label": 1},
    {"text": "Campaigns heat up as candidates rally for voter support.", "label": 1},
    {"text": "The new law aims to strengthen cybersecurity measures.", "label": 1},
    {"text": "The prime minister visits a neighboring country for trade talks.", "label": 1},
    {"text": "Protests erupt against the controversial policy decision.", "label": 1},
    {"text": "A landmark decision by the court shapes future legislation.", "label": 1},
    {"text": "The minister highlights plans to improve education systems.", "label": 1},
    {"text": "National security is at the forefront of the political agenda.", "label": 1},
    {"text": "The governor outlines disaster relief efforts after the floods.", "label": 1},
    {"text": "A major treaty is signed to address climate change issues.", "label": 1},
    {"text": "Students prepare for university entrance exams.", "label": 2},
    {"text": "The new study techniques promise better learning outcomes.", "label": 2},
    {"text": "The education board announces changes in the curriculum.", "label": 2},
    {"text": "A groundbreaking study explores the effects of online learning.", "label": 2},
    {"text": "A new scholarship program supports underprivileged students.", "label": 2},
    {"text": "Teachers attend workshops on integrating technology in classrooms.", "label": 2},
    {"text": "The library adds hundreds of new academic resources.", "label": 2},
    {"text": "Researchers publish findings on improving memory retention.", "label": 2},
    {"text": "Study groups form to tackle challenging science topics.", "label": 2},
    {"text": "Students collaborate on innovative projects for the science fair.", "label": 2},
    {"text": "The school introduces coding classes for young learners.", "label": 2},
    {"text": "An education conference discusses the future of AI in learning.", "label": 2},
    {"text": "Experts share tips for mastering difficult math concepts.", "label": 2},
    {"text": "The university announces free courses on climate studies.", "label": 2},
    {"text": "A recent survey highlights the need for mental health support in schools.", "label": 2},
    {"text": "The tutoring center reports an increase in attendance.", "label": 2},
    {"text": "Students celebrate as their project wins the national innovation award.", "label": 2},
    {"text": "A new app helps students track their study habits effectively.", "label": 2},
]

# Split the data into training and validation sets
train_texts, val_texts = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = Dataset.from_list(train_texts)
val_dataset = Dataset.from_list(val_texts)

# Load Tokenizer and Model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Load DistilBERT model for sequence classification
# Setting `num_labels=3` for 3-class classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# LoRA Configuration (Adapter-Based Fine-Tuning)
# LoRA introduces lightweight adapters in specified layers of the model to fine-tune efficiently
lora_config = LoraConfig(
    r=8,  # Rank of the low-rank adapter matrices
    lora_alpha=16,  # Scaling factor for adapter updates
    target_modules=[
        "attention.q_lin", "attention.k_lin",
        "attention.v_lin", "attention.out_lin"
    ],  # LoRA targets the attention layers of DistilBERT
    task_type="SEQ_CLS",  # Task type is Sequence Classification
)

# Apply LoRA to the model
# This transforms the base model into an adapter-based model for fine-tuning
model = get_peft_model(model, lora_config)

# Tokenization Function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=512)

# Tokenize the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Remove text column after tokenization
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_val = tokenized_val.remove_columns(["text"])

# Set format to PyTorch tensors
tokenized_train.set_format("torch")
tokenized_val.set_format("torch")

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results_peft_distilbert",  # Output directory for model and logs
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    learning_rate=2e-5,  # Learning rate for the optimizer
    per_device_train_batch_size=4,  # Training batch size
    per_device_eval_batch_size=4,  # Evaluation batch size
    num_train_epochs=3,  # Number of training epochs
    weight_decay=0.01,  # Weight decay for regularization
    logging_dir="./logs_peft_distilbert",  # Directory for training logs
    logging_steps=10,  # Log every 10 steps
    no_cuda=True,  # Ensure training runs on CPU
)

# Trainer for fine-tuning the model
trainer = Trainer(
    model=model,  # Adapter-based model with LoRA applied
    args=training_args,  # Training arguments
    train_dataset=tokenized_train,  # Training dataset
    eval_dataset=tokenized_val,  # Validation dataset
    tokenizer=tokenizer,  # Tokenizer
)

# Train the model
trainer.train()

# Save the fine-tuned adapter-based model and tokenizer
model.save_pretrained("./adapter_distilbert_model")
tokenizer.save_pretrained("./adapter_distilbert_model")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1590: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
<ipython-input-3-2903e6c8ae99>:121: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.090300,1.079828
2,1.062100,1.074713
3,1.076700,1.073311


('./adapter_distilbert_model/tokenizer_config.json',
 './adapter_distilbert_model/special_tokens_map.json',
 './adapter_distilbert_model/vocab.txt',
 './adapter_distilbert_model/added_tokens.json')

In [6]:
# Prepare Validation Data
val_texts = [item["text"] for item in val_dataset]
val_labels = [item["label"] for item in val_dataset]

# Tokenize Validation Data
tokenized_val = tokenize_function_eval({"text": val_texts})

# Perform Evaluation
def evaluate(model, tokenized_val, val_labels):
    # Move tensors to the appropriate device (CPU in this case)
    input_ids = tokenized_val["input_ids"]
    attention_mask = tokenized_val["attention_mask"]
    labels = torch.tensor(val_labels)

    with torch.no_grad():
        # Obtain model predictions
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

    # Calculate accuracy
    accuracy = accuracy_score(labels.numpy(), predictions.numpy())

    # Generate classification report
    report = classification_report(labels.numpy(), predictions.numpy(), target_names=["Sports", "Politics", "Education"])

    return accuracy, report

# Run evaluation
accuracy, report = evaluate(model, tokenized_val, val_labels)

# Print Evaluation Results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n")
print(report)


Accuracy: 0.4000

Classification Report:

              precision    recall  f1-score   support

      Sports       0.00      0.00      0.00         1
    Politics       0.00      0.00      0.00         5
   Education       0.40      1.00      0.57         4

    accuracy                           0.40        10
   macro avg       0.13      0.33      0.19        10
weighted avg       0.16      0.40      0.23        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
